In [1]:
from models.model import ResNet18
from models.utils import train_step, eval_step, DataLoaders
from models.transforms import transforms_resnet
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

# Define the transforms for the dataset
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize all images to the same size
    transforms.ToTensor()           # Convert images to PyTorch tensors
])

# Load the dataset
dataset = datasets.ImageFolder(root='./train', transform=transform)

# Split dataset into training (80%) and testing (20%)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for both train and test sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [3]:
BATCH_SIZE = 64
N_EPOCHS = 30

In [4]:
# Train and then Evaluate (Three Different Train and Evaluation Loops)
from tqdm import tqdm
def train_and_eval(train_loader,test_loader,model,loss_fn,optimizer,device,modeltype):
    tr_metric = {"Accuracy":[],"Loss":[]}
    ts_metric = {"Accuracy":[],"Loss":[]}

    for epoch in tqdm(range(N_EPOCHS)):
        tr_loss, tr_acc = train_step(model,train_loader,loss_fn,optimizer,device,modeltype, data="texture")
        ts_loss, ts_acc = eval_step(model,test_loader,loss_fn,device,modeltype,data="texture")

        tr_metric["Accuracy"].append(tr_acc)
        tr_metric["Loss"].append(tr_loss)

        ts_metric["Accuracy"].append(ts_acc)
        ts_metric["Loss"].append(ts_loss)
    
    return tr_metric, ts_metric

model = ResNet18(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,model,loss_func,optimizer,device,'resnet18')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])

100%|██████████| 30/30 [05:00<00:00, 10.02s/it]

Final Train Accuracy: 0.984375
Final Test Accuracy: 0.875
Final Train Loss: 0.11726105604320765
Final Test Loss: 0.41407338380813596


In [5]:
import pandas as pd
pd.DataFrame(tr).to_csv("resnet18_newdataset_tr_1.csv") 
pd.DataFrame(ts).to_csv("resnet18_newdataset_ts_1.csv")

In [6]:
torch.save(model.state_dict(),"resnet18_newdataset.pth")
model.load_state_dict(torch.load("resnet18_newdataset.pth"))   

<All keys matched successfully>